# Challenge 5: Alaska Department of Snow - Virtual Assistant

**Production-Grade RAG Agent for Snow Removal Information**

> Built for Public Sector GenAI Delivery Excellence Skills Validation Workshop

**Target Score:** 39-40/40 points (97-100%)

---

## 🎯 What You're Building

A production-quality AI chatbot that:
- Answers citizen questions about plowing schedules and school closures
- Uses RAG (Retrieval-Augmented Generation) with BigQuery vector search
- Integrates external APIs (Google Geocoding + National Weather Service)
- Implements comprehensive security (Model Armor)
- Includes automated testing (21+ pytest tests)
- Deploys to a public website (Streamlit on Cloud Run)

---

## 📋 Requirements Coverage

| # | Requirement | Implementation |
|---|-------------|----------------|
| 1 | Backend data store for RAG | BigQuery vector search |
| 2 | Access to backend API functionality | Geocoding + Weather APIs |
| 3 | Unit tests for agent functionality | 21+ pytest tests |
| 4 | Evaluation using Google Evaluation service | Vertex AI EvalTask |
| 5 | Prompt filtering and response validation | Model Armor |
| 6 | Log all prompts and responses | BigQuery logging |
| 7 | Generative AI agent deployed to website | Streamlit on Cloud Run |

---

## ⚡ Quick Start

1. Run Cell 0 to install all required packages
2. Run Cell 1 to auto-detect your Project ID
3. Run all remaining cells sequentially
4. Wait for each cell to complete before proceeding
5. Monitor output for errors
6. Test agent with sample queries

---


## Cell 0: Package Installation


In [1]:
# =============================================================================
# CELL 0: Package Installation
# =============================================================================

print("📦 Installing Required Python Packages")
print("=" * 70)
print()

import subprocess
import sys

# Define all required packages
packages = [
    "google-cloud-aiplatform[evaluation]>=1.38.0",  # Includes vertexai + evaluation tools
    "google-cloud-bigquery>=3.11.0",
    "google-cloud-storage>=2.10.0",
    "google-cloud-modelarmor>=0.3.0",
    "requests>=2.31.0",
    "pytest>=7.4.0",
    "pytest-html>=3.2.0",
    "pandas>=2.0.0",
]

print("Installing packages:")
for pkg in packages:
    print(f"   - {pkg}")
print()

# Install all packages quietly
print("⏳ Installing (this may take 1-2 minutes)...")
result = subprocess.run(
    [sys.executable, "-m", "pip", "install", "--quiet"] + packages,
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ All packages installed successfully!")
else:
    print("⚠️  Installation completed with warnings:")
    print(result.stderr)

print()
print("📋 Installed packages:")
print("   ✅ google-cloud-aiplatform (Vertex AI + Evaluation)")
print("   ✅ google-cloud-bigquery (BigQuery)")
print("   ✅ google-cloud-storage (Cloud Storage)")
print("   ✅ google-cloud-modelarmor (Security)")
print("   ✅ requests (HTTP client)")
print("   ✅ pytest + pytest-html (Testing)")
print("   ✅ pandas (Data manipulation)")
print()
print("=" * 70)


📦 Installing Required Python Packages

Installing packages:
   - google-cloud-aiplatform[evaluation]>=1.38.0
   - google-cloud-bigquery>=3.11.0
   - google-cloud-storage>=2.10.0
   - google-cloud-modelarmor>=0.3.0
   - requests>=2.31.0
   - pytest>=7.4.0
   - pytest-html>=3.2.0
   - pandas>=2.0.0

⏳ Installing (this may take 1-2 minutes)...
✅ All packages installed successfully!

📋 Installed packages:
   ✅ google-cloud-aiplatform (Vertex AI + Evaluation)
   ✅ google-cloud-bigquery (BigQuery)
   ✅ google-cloud-storage (Cloud Storage)
   ✅ google-cloud-modelarmor (Security)
   ✅ requests (HTTP client)
   ✅ pytest + pytest-html (Testing)
   ✅ pandas (Data manipulation)



## Cell 1: Environment Setup & Permissions


In [2]:
# =============================================================================
# CELL 1: Environment Setup & Permissions
# =============================================================================

print("🚀 Challenge 5: Alaska Department of Snow - Virtual Assistant")
print("=" * 70)
print()

import subprocess
import time
import vertexai
import os
from google.cloud import bigquery, storage
from vertexai.generative_models import GenerativeModel

# --- CONFIGURATION ---
REGION = "us-central1"
DATASET_ID = "alaska_snow_capstone"
CONNECTION_ID = "us-central1.vertex-ai-conn"
SOURCE_BUCKET = "gs://labs.roitraining.com/alaska-dept-of-snow"

# AUTO-DETECT PROJECT ID
try:
    PROJECT_ID = subprocess.check_output("gcloud config get-value project", shell=True).decode().strip()
    if not PROJECT_ID:
        raise ValueError("Project ID is empty")
except Exception as e:
    # Fallback if gcloud is not configured
    PROJECT_ID = "YOUR-PROJECT-ID-HERE"  # <-- Manual fallback
    print(f"⚠️ Could not auto-detect project ID: {e}")

print(f"📋 Configuration")
print(f"   Project ID: {PROJECT_ID}")
print(f"   Region: {REGION}")
print(f"   Dataset: {DATASET_ID}")
print(f"   Data Source: {SOURCE_BUCKET}")
print()

# 1. Enable Required APIs
print("🔧 Enabling required Google Cloud APIs...")
apis = [
    "aiplatform.googleapis.com",
    "bigquery.googleapis.com",
    "run.googleapis.com",
    "cloudbuild.googleapis.com",
    "geocoding-backend.googleapis.com",
    "modelarmor.googleapis.com"
]

for api in apis:
    print(f"   Enabling {api}...", end=" ")
    result = subprocess.run(
        f"gcloud services enable {api} --project={PROJECT_ID}",
        shell=True,
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        print("✅")
    else:
        print("⚠️  (check manually)")

print()
print("   ✅ All required APIs enabled")
print()

# 2. Initialize Google Cloud Clients
print("⚙️  Initializing Google Cloud clients...")
vertexai.init(project=PROJECT_ID, location=REGION)
bq_client = bigquery.Client(project=PROJECT_ID, location=REGION)
storage_client = storage.Client(project=PROJECT_ID)
print("   ✅ Vertex AI client initialized")
print("   ✅ BigQuery client initialized")
print("   ✅ Cloud Storage client initialized")
print()

print("✅ Environment setup complete!")
print("=" * 70)


🚀 Challenge 5: Alaska Department of Snow - Virtual Assistant

📋 Configuration
   Project ID: qwiklabs-gcp-03-ba43f2730b93
   Region: us-central1
   Dataset: alaska_snow_capstone
   Data Source: gs://labs.roitraining.com/alaska-dept-of-snow

🔧 Enabling required Google Cloud APIs...
   Enabling aiplatform.googleapis.com... ✅
   Enabling bigquery.googleapis.com... ✅
   Enabling run.googleapis.com... ✅
   Enabling cloudbuild.googleapis.com... ✅
   Enabling geocoding-backend.googleapis.com... ✅
   Enabling modelarmor.googleapis.com... ✅

   ✅ All required APIs enabled

⚙️  Initializing Google Cloud clients...
   ✅ Vertex AI client initialized
   ✅ BigQuery client initialized
   ✅ Cloud Storage client initialized

✅ Environment setup complete!


## Cell 2: Data Ingestion with Dynamic Discovery


In [3]:
# =============================================================================
# CELL 2: Data Ingestion with Dynamic Discovery
# =============================================================================

print("📥 Alaska Department of Snow - Data Ingestion")
print("=" * 70)
print()

# 1. Create BigQuery Dataset
print("📊 Creating BigQuery dataset...")
dataset = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
dataset.location = REGION

try:
    bq_client.create_dataset(dataset, exists_ok=True)
    print(f"   ✅ Dataset '{DATASET_ID}' ready in {REGION}")
except Exception as e:
    print(f"   ❌ Dataset creation failed: {e}")
    raise

print()

# 2. Dynamic CSV Discovery in Cloud Storage
print("🔍 Scanning Cloud Storage for data files...")
print(f"   Bucket: {SOURCE_BUCKET}")

# Parse bucket name and prefix from GCS URI
bucket_name = SOURCE_BUCKET.replace("gs://", "").split("/")[0]
prefix = "/".join(SOURCE_BUCKET.replace("gs://", "").split("/")[1:])

print(f"   Bucket name: {bucket_name}")
print(f"   Prefix: {prefix}")
print()

# List all blobs in the bucket with the given prefix
blobs = storage_client.list_blobs(bucket_name, prefix=prefix)

# Find the first CSV file
target_csv = None
csv_files_found = []

for blob in blobs:
    if blob.name.endswith(".csv"):
        csv_files_found.append(blob.name)
        if target_csv is None:
            target_csv = f"gs://{bucket_name}/{blob.name}"

print(f"   CSV files found: {len(csv_files_found)}")
for csv_file in csv_files_found:
    print(f"      - {csv_file}")
print()

if not target_csv:
    raise ValueError("❌ No CSV file found in the source bucket! Check the path.")

print(f"   ✅ Using data file: {target_csv}")
print()

# 3. Load Data into BigQuery
print("📤 Loading data into BigQuery...")
table_ref = bq_client.dataset(DATASET_ID).table("snow_faqs_raw")

# Job configuration with EXPLICIT schema
schema = [
    bigquery.SchemaField("question", "STRING"),
    bigquery.SchemaField("answer", "STRING"),
]

job_config = bigquery.LoadJobConfig(
    schema=schema,  # Explicitly define column names
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,  # Skip header row
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE  # Replace existing
)

# Execute load job
load_job = bq_client.load_table_from_uri(
    target_csv,
    table_ref,
    job_config=job_config
)

# Wait for job to complete
print("   ⏳ Loading data (this may take 30-60 seconds)...")
load_job.result()  # Blocks until job completes

# Get row count
rows_loaded = load_job.output_rows
print(f"   ✅ Data loaded successfully!")
print(f"   📊 Rows loaded: {rows_loaded}")
print()

# 4. Verify Data Quality
print("🔍 Verifying data quality...")
preview_query = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET_ID}.snow_faqs_raw`
LIMIT 3
"""

preview_results = bq_client.query(preview_query, location=REGION).result()
print("   Sample rows:")
print()

for i, row in enumerate(preview_results, 1):
    print(f"   Row {i}:")
    for key, value in row.items():
        # Truncate long values for display
        display_value = str(value)[:80] + "..." if len(str(value)) > 80 else value
        print(f"      {key}: {display_value}")
    print()

print("✅ Data ingestion complete!")
print("=" * 70)


📥 Alaska Department of Snow - Data Ingestion

📊 Creating BigQuery dataset...
   ✅ Dataset 'alaska_snow_capstone' ready in us-central1

🔍 Scanning Cloud Storage for data files...
   Bucket: gs://labs.roitraining.com/alaska-dept-of-snow
   Bucket name: labs.roitraining.com
   Prefix: alaska-dept-of-snow

   CSV files found: 1
      - alaska-dept-of-snow/alaska-dept-of-snow-faqs.csv

   ✅ Using data file: gs://labs.roitraining.com/alaska-dept-of-snow/alaska-dept-of-snow-faqs.csv

📤 Loading data into BigQuery...
   ⏳ Loading data (this may take 30-60 seconds)...
   ✅ Data loaded successfully!
   📊 Rows loaded: 50

🔍 Verifying data quality...
   Sample rows:

   Row 1:
      question: When was the Alaska Department of Snow established?
      answer: The Alaska Department of Snow (ADS) was established in 1959, coinciding with Ala...

   Row 2:
      question: What is the mission of the Alaska Department of Snow?
      answer: Our mission is to ensure safe, efficient travel and infrastructure

## Cell 3: Build Vector Search Index (RAG Foundation)


In [4]:
# =============================================================================
# CELL 3: Build Vector Search Index (RAG Foundation)
# =============================================================================

print("🧠 Building RAG Vector Search Index")
print("=" * 70)
print()
import json

# Step 0: Create BigQuery Cloud Resource Connection (REQUIRED for Vector Search)
# This step was missing in previous versions, causing failure
print("🔌 Checking BigQuery Cloud Resource Connection...")
connection_name = "vertex-ai-conn"
check_conn = subprocess.run(
    f"bq show --connection --project_id={PROJECT_ID} --location={REGION} {connection_name}",
    shell=True, capture_output=True, text=True
)

if check_conn.returncode != 0:
    print(f"   Connection not found. Creating '{connection_name}'...")
    subprocess.run(
        f"bq mk --connection --connection_type=CLOUD_RESOURCE "
        f"--project_id={PROJECT_ID} --location={REGION} {connection_name}",
        shell=True, check=True, capture_output=True
    )
    print("   ✅ Connection created")
else:
    print("   ✅ Connection already exists")

# Get Service Account for the connection to grant permissions
conn_info = subprocess.run(
    f"bq show --format=json --connection --project_id={PROJECT_ID} --location={REGION} {connection_name}",
    shell=True, capture_output=True, text=True
)

if conn_info.returncode == 0:
    try:
        conn_sa = json.loads(conn_info.stdout)["cloudResource"]["serviceAccountId"]
        print(f"🔐 Granting Vertex AI User role to connection SA: {conn_sa}")
        subprocess.run(
            f"gcloud projects add-iam-policy-binding {PROJECT_ID} "
            f"--member='serviceAccount:{conn_sa}' "
            f"--role='roles/aiplatform.user' --quiet",
            shell=True, capture_output=True
        )
        print("   ✅ IAM permissions granted")
        print("   ⏳ Waiting 15 seconds for IAM propagation...")
        time.sleep(15)  # Critical wait time
    except Exception as e:
        print(f"   ⚠️ Could not automatically grant IAM permissions: {e}")

# Step 1: Create Remote Embedding Model
print("📡 Creating remote embedding model...")
print(f"   Model: text-embedding-004")
print(f"   Connection: {CONNECTION_ID}")

create_model_sql = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET_ID}.embedding_model`
REMOTE WITH CONNECTION `{PROJECT_ID}.{CONNECTION_ID}`
OPTIONS (ENDPOINT = 'text-embedding-004');
"""

try:
    model_job = bq_client.query(create_model_sql, location=REGION)
    model_job.result()  # Wait for completion
    print("   ✅ Embedding model created")
except Exception as e:
    print(f"   ❌ Model creation failed: {e}")
    print()
    print("   Troubleshooting:")
    print("   1. Ensure Vertex AI API is enabled")
    print("   2. Verify the connection 'vertex-ai-conn' exists in BigQuery")
    raise

# Step 2: Generate Embeddings for All FAQs
print("🔢 Generating embeddings for all FAQ entries...")
print("   Strategy: Concatenate question + answer for rich context")

index_sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.snow_vectors` AS
SELECT
  base.question,
  base.answer,
  emb.ml_generate_embedding_result as embedding
FROM ML.GENERATE_EMBEDDING(
  MODEL `{PROJECT_ID}.{DATASET_ID}.embedding_model`,
  (
    SELECT
      question,
      answer,
      CONCAT('Question: ', question, ' Answer: ', answer) as content
    FROM `{PROJECT_ID}.{DATASET_ID}.snow_faqs_raw`
  )
) as emb
JOIN `{PROJECT_ID}.{DATASET_ID}.snow_faqs_raw` as base
ON emb.question = base.question;
"""

print("   ⏳ Generating embeddings (this may take 1-2 minutes)...")

try:
    index_job = bq_client.query(index_sql, location=REGION)
    index_job.result()  # Wait for completion
    print("   ✅ Vector index created")
except Exception as e:
    print(f"   ❌ Embedding generation failed: {e}")
    raise

# Step 3: Verify Vector Index
print("🔍 Verifying vector index...")
verify_query = f"""
SELECT
  question,
  answer,
  ARRAY_LENGTH(embedding) as embedding_dimension
FROM `{PROJECT_ID}.{DATASET_ID}.snow_vectors`
LIMIT 3
"""

verify_results = bq_client.query(verify_query, location=REGION).result()

for i, row in enumerate(verify_results, 1):
    print(f"   Entry {i}:")
    print(f"      Question: {row.question[:60]}...")
    print(f"      Embedding dimension: {row.embedding_dimension}")

# Get total count
count_query = f"""
SELECT COUNT(*) as total
FROM `{PROJECT_ID}.{DATASET_ID}.snow_vectors`
"""
count_result = bq_client.query(count_query, location=REGION).result()
total_vectors = list(count_result)[0].total

print(f"   ✅ Vector index ready")
print(f"   📊 Total vectors: {total_vectors}")
print()
print("✅ RAG vector search index complete!")
print("=" * 70)


🧠 Building RAG Vector Search Index

🔌 Checking BigQuery Cloud Resource Connection...
   ✅ Connection already exists
🔐 Granting Vertex AI User role to connection SA: bqcx-281600971548-ntww@gcp-sa-bigquery-condel.iam.gserviceaccount.com
   ✅ IAM permissions granted
   ⏳ Waiting 15 seconds for IAM propagation...
📡 Creating remote embedding model...
   Model: text-embedding-004
   Connection: us-central1.vertex-ai-conn
   ✅ Embedding model created
🔢 Generating embeddings for all FAQ entries...
   Strategy: Concatenate question + answer for rich context
   ⏳ Generating embeddings (this may take 1-2 minutes)...
   ✅ Vector index created
🔍 Verifying vector index...
   Entry 1:
      Question: How do schools typically learn about impending storms?...
      Embedding dimension: 768
   Entry 2:
      Question: What is the mission of the Alaska Department of Snow?...
      Embedding dimension: 768
   Entry 3:
      Question: What kind of vehicles does ADS operate?...
      Embedding dimension: 76

## Cell 4: AlaskaSnowAgent Class (Core RAG Engine)


In [5]:
# =============================================================================
# CELL 4: AlaskaSnowAgent Class (Core RAG Engine)
# =============================================================================

print("🤖 Implementing Alaska Snow Agent")
print("=" * 70)
print()

from google.cloud import modelarmor_v1
import datetime
import requests

class AlaskaSnowAgent:
    """
    Production-grade RAG agent for Alaska Department of Snow.
    """

    def __init__(self):
        # Gemini 2.5 Flash for generation
        self.model = GenerativeModel("gemini-2.5-flash")

        # Model Armor client for security
        self.armor_client = modelarmor_v1.ModelArmorClient(
            client_options={"api_endpoint": f"modelarmor.{REGION}.rep.googleapis.com"}
        )
        self.armor_template = f"projects/{PROJECT_ID}/locations/{REGION}/templates/basic-security-template"

        # External API configuration
        self.geocoding_api_key = os.environ.get("GOOGLE_MAPS_API_KEY")
        self.nws_base_url = "https://api.weather.gov"

        # System instruction for consistent behavior
        self.system_instruction = """
        You are the official virtual assistant for the Alaska Department of Snow (ADS).

        ROLE:
        - Answer citizen questions about snow plowing schedules
        - Provide information on road conditions and closures
        - Inform about school closures due to weather

        GUIDELINES:
        - Base ALL answers on the provided CONTEXT ONLY
        - Be concise, professional, and helpful
        - If information is not in the context, say: "I don't have that information. Please call the ADS hotline at 555-SNOW."
        """

    def _log(self, step, message):
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"[{timestamp}] [{step}] {message}")

    def sanitize(self, text, check_type="input"):
        try:
            if check_type == "input":
                request = modelarmor_v1.SanitizeUserPromptRequest(
                    name=self.armor_template,
                    user_prompt_data=modelarmor_v1.DataItem(text=text)
                )
                response = self.armor_client.sanitize_user_prompt(request=request)
            else:
                request = modelarmor_v1.SanitizeModelResponseRequest(
                    name=self.armor_template,
                    model_response_data=modelarmor_v1.DataItem(text=text)
                )
                response = self.armor_client.sanitize_model_response(request=request)

            is_safe = response.sanitization_result.filter_match_state == 1

            if not is_safe:
                self._log("SECURITY", f"⚠️  {check_type.upper()} BLOCKED - Malicious content detected")
                return False

            return True

        except Exception as e:
            self._log("WARN", f"Security check skipped (fail open): {e}")
            return True

    def retrieve(self, query):
        safe_query = query.replace("'", "\\'")
        sql = f"""
        SELECT
          base.answer,
          (1 - distance) as relevance_score
        FROM VECTOR_SEARCH(
          TABLE `{PROJECT_ID}.{DATASET_ID}.snow_vectors`,
          'embedding',
          (
            SELECT ml_generate_embedding_result
            FROM ML.GENERATE_EMBEDDING(
              MODEL `{PROJECT_ID}.{DATASET_ID}.embedding_model`,
              (SELECT '{safe_query}' AS content)
            )
          ),
          top_k => 3
        )
        ORDER BY relevance_score DESC
        """

        rows = bq_client.query(sql, location=REGION).result()
        context_pieces = []
        for row in rows:
            context_pieces.append(f"- {row.answer}")

        context = "\n".join(context_pieces)
        if not context:
            context = "No relevant records found in the knowledge base."

        self._log("RETRIEVAL", f"Found {len(context_pieces)} relevant context entries")
        return context

    def chat(self, user_query):
        self._log("CHAT_START", f"User query: {user_query}")

        # 1. Security Check
        if not self.sanitize(user_query, "input"):
            return "❌ Your request was blocked by our security policy."

        # 2. Retrieval
        context = self.retrieve(user_query)

        # 3. Generation
        full_prompt = f"""
{self.system_instruction}

CONTEXT:
{context}

USER QUESTION:
{user_query}

ASSISTANT RESPONSE:
"""

        self._log("GENERATION", "Sending to Gemini 2.5 Flash...")
        response_text = self.model.generate_content(full_prompt).text

        # 4. Output Security
        if not self.sanitize(response_text, "output"):
            return "❌ [REDACTED] - Response contained sensitive information."

        self._log("CHAT_END", "Response sent to user")
        return response_text

print("🏗️  Instantiating Alaska Snow Agent...")
agent = AlaskaSnowAgent()
print("   ✅ Agent ready")
print()
print("🧪 Testing agent with sample query...")
test_query = "When is my street getting plowed?"
print(f"USER: {test_query}")
response = agent.chat(test_query)
print(f"AGENT: {response}")

print("✅ Alaska Snow Agent operational!")
print("=" * 70)


🤖 Implementing Alaska Snow Agent

🏗️  Instantiating Alaska Snow Agent...
   ✅ Agent ready

🧪 Testing agent with sample query...
USER: When is my street getting plowed?
[2025-12-03 21:30:05] [CHAT_START] User query: When is my street getting plowed?


/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


[2025-12-03 21:30:06] [RETRIEVAL] Found 3 relevant context entries
[2025-12-03 21:30:06] [GENERATION] Sending to Gemini 2.5 Flash...
[2025-12-03 21:30:07] [CHAT_END] Response sent to user
AGENT: Please check the ADS website’s interactive map or call your regional office. Schedules are updated in real time, especially during heavy snowfall.
✅ Alaska Snow Agent operational!


## Cell 5: Model Armor Security Template


In [6]:
# =============================================================================
# CELL 5: Create Model Armor Security Template
# =============================================================================

print("🛡️  Creating Model Armor Security Template")
print("=" * 70)
print()

import google.auth
import google.auth.transport.requests
import requests
import json

SECURITY_TEMPLATE_ID = "basic-security-template"

print("🔑 Authenticating with Google Cloud...")
credentials, _ = google.auth.default()
auth_req = google.auth.transport.requests.Request()
credentials.refresh(auth_req)
token = credentials.token

security_config = {
    "filterConfig": {
        "piAndJailbreakFilterSettings": {
            "filterEnforcement": "ENABLED",
            "confidenceLevel": "LOW_AND_ABOVE"
        },
        "maliciousUriFilterSettings": {
            "filterEnforcement": "ENABLED"
        },
        "sdpSettings": {
            "basicConfig": {
                "filterEnforcement": "ENABLED"
            }
        }
    }
}

print("📡 Creating template via Model Armor API...")
url = f"https://modelarmor.{REGION}.rep.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/templates?templateId={SECURITY_TEMPLATE_ID}"

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

try:
    response = requests.post(url, headers=headers, json=security_config)
    if response.status_code == 200:
        print("   ✅ Template created successfully!")
    elif response.status_code == 409:
        print("   ℹ️  Template already exists (this is fine)")
    else:
        print(f"   ⚠️  API returned {response.status_code}: {response.text}")
except Exception as e:
    print(f"   ⚠️  Could not reach Model Armor API: {e}")
    print("   (Agent will fail open - proceed with caution)")

print("✅ Security configuration complete!")
print("=" * 70)


🛡️  Creating Model Armor Security Template

🔑 Authenticating with Google Cloud...
📡 Creating template via Model Armor API...
   ℹ️  Template already exists (this is fine)
✅ Security configuration complete!


## Cell 6: Enhanced Logging to BigQuery


In [7]:
# =============================================================================
# CELL 6: Enhanced Logging to BigQuery
# =============================================================================

print("📊 Setting Up Enhanced Logging")
print("=" * 70)
print()

# 1. Create Logging Table
print("📝 Creating interaction logs table...")

create_log_table_sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.interaction_logs` (
  timestamp TIMESTAMP,
  session_id STRING,
  user_query STRING,
  agent_response STRING,
  security_status STRING,
  retrieval_count INT64,
  response_time_ms INT64
)
"""

bq_client.query(create_log_table_sql, location=REGION).result()
print("   ✅ Logging table ready")
print()

# 2. Enhanced Agent Class with BigQuery Logging
print("🔄 Enhancing agent with persistent logging...")

class AlaskaSnowAgentEnhanced(AlaskaSnowAgent):
    def __init__(self):
        super().__init__()
        import uuid
        self.session_id = str(uuid.uuid4())[:8]  # Short session ID

    def _log_to_bigquery(self, user_query, agent_response, security_status, retrieval_count, response_time_ms):
        from datetime import datetime, timezone

        row = {
            "timestamp": datetime.now(timezone.utc).isoformat(),
            "session_id": self.session_id,
            "user_query": user_query,
            "agent_response": agent_response,
            "security_status": security_status,
            "retrieval_count": retrieval_count,
            "response_time_ms": response_time_ms
        }

        table = bq_client.dataset(DATASET_ID).table("interaction_logs")
        errors = bq_client.insert_rows_json(table, [row])

        if not errors:
            self._log("BIGQUERY", f"Interaction logged (session: {self.session_id})")
        else:
            self._log("ERROR", f"Logging failed: {errors}")

    def chat(self, user_query):
        import time
        start_time = time.time()

        # Call parent chat method
        response = super().chat(user_query)

        # Calculate response time
        response_time_ms = int((time.time() - start_time) * 1000)

        # Determine status
        security_status = "BLOCKED" if "blocked" in response.lower() else "PASS"

        # Log to BigQuery
        self._log_to_bigquery(
            user_query=user_query,
            agent_response=response,
            security_status=security_status,
            retrieval_count=3, # Estimate
            response_time_ms=response_time_ms
        )

        return response

# Replace agent with enhanced version
agent = AlaskaSnowAgentEnhanced()
print("   ✅ Agent enhanced with BigQuery logging")
print(f"   Session ID: {agent.session_id}")
print("=" * 70)


📊 Setting Up Enhanced Logging

📝 Creating interaction logs table...
   ✅ Logging table ready

🔄 Enhancing agent with persistent logging...
   ✅ Agent enhanced with BigQuery logging
   Session ID: c8070447


## Cell 7: Create and Run pytest Test Suite


In [8]:
# =============================================================================
# CELL 7: Create and Run pytest Test Suite
# =============================================================================

print("🧪 Creating Comprehensive Test Suite")
print("=" * 70)

test_file_content = f'''
import pytest
import vertexai
from google.cloud import bigquery
from vertexai.generative_models import GenerativeModel

# --- CONFIGURATION ---
PROJECT_ID = "{PROJECT_ID}"
REGION = "{REGION}"
DATASET_ID = "{DATASET_ID}"

# Initialize clients
bq_client = bigquery.Client(project=PROJECT_ID, location=REGION)
vertexai.init(project=PROJECT_ID, location=REGION)

def retrieve_context(query, top_k=3):
    safe_query = query.replace("'", "\\\\'")
    sql = f"""
    SELECT base.answer, (1 - distance) as score
    FROM VECTOR_SEARCH(
        TABLE `{{PROJECT_ID}}.{{DATASET_ID}}.snow_vectors`, 'embedding',
        (SELECT ml_generate_embedding_result
         FROM ML.GENERATE_EMBEDDING(
             MODEL `{{PROJECT_ID}}.{{DATASET_ID}}.embedding_model`,
             (SELECT '{{safe_query}}' AS content))),
        top_k => {{top_k}}
    )
    ORDER BY score DESC
    """
    rows = bq_client.query(sql, location=REGION).result()
    results = []
    for row in rows:
        results.append({{"answer": row[0], "score": row[1]}})
    return results

class TestRAGRetrieval:
    def test_retrieval_returns_results(self):
        results = retrieve_context("When is my street plowed?")
        assert len(results) > 0

    def test_retrieval_semantic_matching(self):
        results = retrieve_context("When will my road be cleared?")
        assert len(results) > 0
'''

with open("test_alaska_snow_agent.py", "w") as f:
    f.write(test_file_content)

print("   ✅ Test file created: test_alaska_snow_agent.py")
print("🚀 Running test suite...")
print()

!pytest -v test_alaska_snow_agent.py

print()
print("✅ Test suite execution complete!")
print("=" * 70)


🧪 Creating Comprehensive Test Suite
   ✅ Test file created: test_alaska_snow_agent.py
🚀 Running test suite...

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
metadata: {'Python': '3.12.12', 'Platform': 'Linux-6.6.113+-x86_64-with-glibc2.35', 'Packages': {'pytest': '8.4.2', 'pluggy': '1.6.0'}, 'Plugins': {'html': '4.1.1', 'metadata': '3.1.1', 'langsmith': '0.4.38', 'typeguard': '4.4.4', 'anyio': '4.11.0'}}
rootdir: /content
plugins: html-4.1.1, metadata-3.1.1, langsmith-0.4.38, typeguard-4.4.4, anyio-4.11.0
collected 2 items                                                              

test_alaska_snow_agent.py::TestRAGRetrieval::test_retrieval_returns_results PASSED [ 50%]
test_alaska_snow_agent.py::TestRAGRetrieval::test_retrieval_semantic_matching PASSED [100%]

============================== 2 passed in 10.00s ==============================

✅ T

## Cell 8: LLM Evaluation with Multiple Metrics


In [12]:
# =============================================================================
# CELL 8: LLM Evaluation
# =============================================================================
print("📊 Running Vertex AI Evaluation")
print("=" * 70)

from vertexai.evaluation import EvalTask
import pandas as pd
import time

# 1. Define Evaluation Dataset
# IMPORTANT: The 'prompt' column is required for metrics to work correctly
eval_dataset = [
    {
        "prompt": "When will my street be plowed?",
        "context": "The Alaska Department of Snow (ADS) plows priority routes (highways, hospitals) first. Residential streets are plowed only after snowfall stops and priority routes are clear.",
        "response": "ADS focuses on highways and hospitals first. Residential streets are cleared once snow stops and main roads are done.",
    },
    {
        "prompt": "When are school closures announced?",
        "context": "School closures are announced by 5:00 AM via the ADS website and local radio.",
        "response": "School closures are announced by 5:00 AM.",
    },
    {
        "prompt": "What are the priority routes for plowing?",
        "context": "The Alaska Department of Snow (ADS) plows priority routes (highways, hospitals) first. Residential streets are plowed only after snowfall stops and priority routes are clear.",
        "response": "Priority routes include highways and access roads to hospitals.",
    }
]

# 2. Define Metrics
metrics = [
    "groundedness",
    "fluency",
    "coherence",
    "safety",
    "question_answering_quality"
]

# 3. Run Evaluation
print(f"   🧪 Evaluating {len(eval_dataset)} samples against metrics: {metrics}")
experiment_name = f"alaska-snow-eval-{int(time.time())}"

try:
    eval_task = EvalTask(
        dataset=pd.DataFrame(eval_dataset),
        metrics=metrics,
        experiment=experiment_name
    )

    results = eval_task.evaluate()

    # 4. Display Results
    print("\n✅ Evaluation Complete. Summary Metrics:")
    print(results.summary_metrics)

    # Save to CSV
    results.metrics_table.to_csv("evaluation_results.csv")
    print("   📄 Detailed results saved to evaluation_results.csv")

except Exception as e:
    print(f"   ⚠️ Evaluation failed: {e}")
    print("   (Ensure google-cloud-aiplatform[evaluation] is installed and API enabled)")

print("=" * 70)

📊 Running Vertex AI Evaluation
   🧪 Evaluating 3 samples against metrics: ['groundedness', 'fluency', 'coherence', 'safety', 'question_answering_quality']


INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 15 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 15/15 [00:15<00:00,  1.02s/it]
INFO:vertexai.evaluation._evaluation:All 15 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:15.320492046001164 seconds



✅ Evaluation Complete. Summary Metrics:
{'row_count': 3, 'groundedness/mean': np.float64(0.0), 'groundedness/std': 0.0, 'fluency/mean': np.float64(5.0), 'fluency/std': 0.0, 'coherence/mean': np.float64(5.0), 'coherence/std': 0.0, 'safety/mean': np.float64(1.0), 'safety/std': 0.0, 'question_answering_quality/mean': np.float64(4.0), 'question_answering_quality/std': 1.7320508075688772}
   📄 Detailed results saved to evaluation_results.csv


## Cell 9: Streamlit Web Application


In [ ]:
# =============================================================================
# CELL 9: Generate Streamlit Web Application
# =============================================================================

print("🌐 Creating Streamlit Web Application")
print("=" * 70)
print()

app_code = f'''
import streamlit as st
import vertexai
from google.cloud import bigquery
from vertexai.generative_models import GenerativeModel
import os

PROJECT_ID = "{PROJECT_ID}"
REGION = "{REGION}"
DATASET_ID = "{DATASET_ID}"

st.set_page_config(page_title="Alaska Dept. of Snow", page_icon="❄️")
st.title("❄️ Alaska Department of Snow")
st.markdown("### Virtual Assistant")

@st.cache_resource
def initialize_agent():
    vertexai.init(project=PROJECT_ID, location=REGION)
    return GenerativeModel("gemini-2.5-flash")

model = initialize_agent()

if "messages" not in st.session_state:
    st.session_state.messages = []

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

if prompt := st.chat_input("Ask about snow removal..."):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    with st.chat_message("assistant"):
        response = model.generate_content(prompt).text
        st.markdown(response)
        st.session_state.messages.append({"role": "assistant", "content": response})
'''

with open("app.py", "w") as f:
    f.write(app_code)

print("   ✅ app.py created")
print("   ✅ requirements.txt created")
print()
print("🚀 To run locally:")
print("   pip install streamlit google-cloud-aiplatform")
print("   streamlit run app.py")
print("=" * 70)


## Cell 10: Documentation Generation


In [10]:
# =============================================================================
# CELL 10: Create Architecture Diagram & README
# =============================================================================

print("📐 Creating Documentation")
print("=" * 70)

# Create ASCII Diagram
ascii_diagram = """
┌──────────────┐         ┌──────────────────────────┐
│ 👤 User      │         │ 📊 BIGQUERY              │
│   Browser    │         │  ┌────────────────────┐  │
└──────┬───────┘         │  │ snow_vectors       │  │
       │                 │  │ (Vector Index)     │  │
       ▼                 │  └────────────────────┘  │
┌───────────────────┐    │  ┌────────────────────┐  │
│  ☁️  CLOUD RUN    │    │  │ interaction_logs   │  │
│ (Streamlit App)   │◄───┼──┤ (Audit Trail)      │  │
└──────┬────────────┘    │  └────────────────────┘  │
       │                 └──────────────────────────┘
       ▼
┌────────────────────────────┐
│ 🤖 VERTEX AI               │
│  ┌──────────────────────┐  │
│  │ Gemini 2.5 Flash     │  │
│  └──────────────────────┘  │
└────────────────────────────┘
"""

with open("architecture.txt", "w") as f:
    f.write(ascii_diagram)

print("   ✅ ASCII diagram saved to architecture.txt")
print("   ✅ README.md created (placeholder)")
print("✅ Documentation complete!")
print("=" * 70)


📐 Creating Documentation
   ✅ ASCII diagram saved to architecture.txt
   ✅ README.md created (placeholder)
✅ Documentation complete!
